# Interactive Assistant with Pydantic and Ollama for Dynamic JSON-based Responses  

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>


## Description  

- This application demonstrates an interactive assistant capable of generating structured responses using **Pydantic models** and **Ollama's language model**.  

- It showcases how to validate, process, and generate JSON objects based on user-defined schemas while integrating error handling and a structured environment setup.  

---

## Uses  

- **Validating JSON structures** for various use cases, such as API responses.  

- **Building intelligent assistants** with predefined response formats.  

- **Creating reproducible workflows** in Jupyter Notebooks.  

- **Simplifying interactions** with language models through schema-driven prompts. 


## Step 1: Boilerplate Setup

This step sets up the boilerplate code for the project. It includes:

- ### Import Statements:

    - `os`: Used for running shell commands and accessing environment variables.

    - `load_dotenv`: Loads environment variables from a `.env` file into the system for easy configuration.

    - `clear_output`: Clears the notebook's output to keep it clean after successful setup.



- ### Global Variables:

    - `requirements_installed`: Tracks whether dependencies are already installed.
    
    - `max_retries`: Limits how many times the code will retry installing dependencies in case of failure.
    
    - `REQUIRED_ENV_VARS`: Specifies the environment variables that must be present.



- ### `install_requirements` Function:

    - Uses the `os.system` command to run `pip install -r requirements.txt`.

    - If the installation fails, it retries up to `max_retries`.

    - If retries are exhausted, it exits the program with a failure code.



- ### `setup_env` Function:

    - Loads environment variables from `.env`.

    - Verifies the presence of each required variable using the `check_env` function.

    - Exits the program if any required environment variable is missing.



- ### Execution:

    - Calls `install_requirements` to install dependencies.

    - Calls `setup_env` to validate the environment.

    - Clears the output and confirms the setup is complete.



In [ ]:
# Boilerplate: This block goes into every notebook.
# It sets up the environment, installs the requirements, and checks for the required environment variables.

from IPython.display import clear_output
from dotenv import load_dotenv
import os

requirements_installed = False
max_retries = 3
retries = 0
REQUIRED_ENV_VARS = ["OPENAI_API_KEY"]


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv()

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)


install_requirements()
clear_output()
setup_env()
print("🚀 Setup complete. Continue to the next cell.")

## Step 2: Adding a Simple Tool for Arithmetic and Using Ollama Chat for Model Integration  

---

### 1. Function Definition: **`add_two_numbers`**  

#### **Definition**  

- **`def add_two_numbers(a: int, b: int) -> int:`**  

  - Defines a function named **`add_two_numbers`** that takes two arguments, **`a`** and **`b`**.  

  - Both arguments are annotated as integers (**`int`**), and the return type is also expected to be an integer.  

#### **Type Annotations**  

- **Purpose**: Improves code readability and assists static type-checking tools.  

#### **Docstring**  

- A multi-line string explaining the function's purpose, arguments, and return value:  

  - **Purpose**: Adds two numbers.  

  - **Arguments**:  

    - **`a`**: First integer to be added.  

    - **`b`**: Second integer to be added.  

  - **Returns**: The sum of **`a`** and **`b`** as an integer.  


#### **Code**  

- **`return a + b`**: Performs the arithmetic addition of the two input integers (**`a`** and **`b`**) and returns the result.  

---

### 2. Calling the Ollama Chat Function  

#### **Code**  

- **`response = ollama.chat(...)`**: Calls the **`ollama.chat`** function to interact with the **`llama3.1`** language model.  

#### **Language Model Version**  

- **`"llama3.1"`**: Specifies the version of the language model to use for the interaction.  

#### **Input Message**  

- **`messages=[{"role": "user", "content": "What is 10 + 10?"}]`**  

  - **`messages`**: A list of dictionaries representing a conversation.  

  - **`"role": "user"`**: Indicates the message comes from the user.  

  - **`"content": "What is 10 + 10?"`**: The user’s query asking the model to compute the sum of 10 and 10.  


#### **Available Tools**  

- **`tools=[add_two_numbers]`**  

  - A list of tools (functions) available to the model during the interaction.  
  
  - Includes the **`add_two_numbers`** function as a callable tool.  

---

### 3. Checking for Tool Calls  

#### **Code**  

- **`if response.message.tool_calls:`**  

  - Checks if the model has made any tool calls in its response.  

#### **Tool Call Details**  

- **`name = response.message.tool_calls[0].tool_name`**  

  - Extracts the **`tool_name`** of the first tool call in the list.  

#### **Output**  

- **`print(f"🦙 The tool name is {name}")`**: Prints the tool name invoked by the model.  

#### **Arguments Extraction**  

- **`args = response.message.tool_calls[0].args`**  

  - Extracts the arguments passed to the tool during the interaction.  

---

### 4. Printing the Tool Arguments  

#### **Code**  

- **`if args:`**: Checks if arguments were included in the tool call.  

#### **Output**  

- **`print(f"🦙 The arguments are {args}")`**: Prints the arguments for the invoked tool.  

---

### 5. Printing the Final Response  

#### **Code**  

- **`print(response)`**: Displays the entire response object returned by **`ollama.chat`**.  

- **`print(response.message.tool_calls)`**: Prints the tool calls made during the interaction for detailed insight.  

---


### Summary
- The code demonstrates an integration between a language model (llama3.1) and a custom function (add_two_numbers) to handle queries requiring arithmetic operations.

- The model intelligently invokes the tool when necessary, passing the required arguments, and the outputs are displayed for validation.

- The approach combines the power of conversational AI with tool-assisted operations, enabling dynamic and interactive solutions.

In [ ]:
import ollama


def add_two_numbers(a: int, b: int) -> int:
    """
    Add two numbers

    Args:
      a: The first integer number
      b: The second integer number

    Returns:
      int: The sum of the two numbers
    """
    return a + b


response = ollama.chat(
    "llama3.1",
    messages=[{"role": "user", "content": "What is 10 + 10?"}],
    tools=[add_two_numbers],  # Actual function reference
)


if response.message.tool_calls:
    name = response.message.tool_calls[0].tool_name
    print(f"🦙 The tool name is {name}")
    args = response.message.tool_calls[0].args

    if args:
        print(f"🦙 The arguments are {args}")


print(response, response.message.tool_calls)

## Step 3: Listing Available Ollama Models

This block retrieves and displays all the available language models provided by Ollama. It ensures the required models are accessible before proceeding with further operations.

In [ ]:
print(f"Listing available models...")

pretty_formatted_list = list(ollama.list())
models = pretty_formatted_list[0][0]
print(pretty_formatted_list)

## Step 4: AI-Powered Structured Response Generation with Pydantic Validation


### 1. Setting Default Parameters and Imports  

#### **Imports**  

- **`from typing import Union`**  

  - Imports the **`Union`** type hint, allowing functions to specify multiple possible return types (e.g., a function may return a specific object or **`None`**).  

- **`from pydantic import BaseModel`**  

  - Imports **`BaseModel`** from **`Pydantic`**, a Python library for data validation and settings management. 

  - **`BaseModel`** is used to define structured data schemas.  

- **`import json`**  

  - Imports the **`json`** library for handling **JSON (JavaScript Object Notation)** data, widely used in APIs.  

- **`import traceback`**  

  - Imports **`traceback`** to print detailed error messages and debug stack traces when exceptions occur.  

---

### 2. Defining Constants  

#### **Constants**  

- **`DEFAULT_SYSTEM_PROMPT`**  

  - A string constant containing a predefined system instruction for the AI model.  

  - Ensures the model remains helpful and consistent in its responses.  

- **`DEFAULT_TEMPERATURE`**  

  - Controls randomness in AI responses.  

  - A lower value like 0.5 produces more focused and deterministic outputs.  

- **`DEFAULT_MAX_TOKENS`**  

  - Specifies the maximum number of tokens (words or parts of words) the model can generate in its response.  

- **`DEFAULT_OLLAMA_MODEL`**  

  - Sets the default model to **`phi4`** for handling requests.  

- **`DEFAULT_VERBOSE` and `DEFAULT_DEBUG`**  

  - Flags that control debugging and verbosity of logging.  

  - **`DEFAULT_VERBOSE`**: Displays general information during execution.  

  - **`DEFAULT_DEBUG`**: Provides more detailed diagnostic information.  


---

### 3. **`build_dummy_pydantic_object`**   

#### **Purpose**  

- Generates a placeholder object using a given **Pydantic** schema.  

- Useful for creating default or mock data based on a schema.  


#### **Parameters**  

- **`schema`**: A **Pydantic** schema class that defines the structure of the object.  

#### **Return Value**  

- An instance of the **Pydantic** schema populated with default values.  

#### **Implementation**  

- Calls the schema class to create an object instance using its default attributes.  

---

### 4. **`generate_object`**   

#### **Purpose**  

- Communicates with the AI model using the provided prompt and a structured response schema.  

#### **Parameters**  

- **`prompt`**: The input query for the AI model.  

- **`response_model`**: A **Pydantic** schema defining the structure of the expected output.

- **`system`, `model`, `temperature`, `max_tokens`**: Optional arguments to customize the AI's behavior.  

- **`debug`, `verbose`**: Flags to control logging and diagnostic output.  


#### **Return Value**  

- Returns a validated **Pydantic** object representing the model's response, or **`None`** if an error occurs.  

---

### 5. Generating and Debugging AI Response  

#### **Purpose**  

- Constructs a detailed prompt for the AI, including the input query and the expected schema.  

#### **Code**  

- **`build_dummy_pydantic_object(response_model).model_dump_json():`**  

  - Converts a dummy object of the response schema into a JSON string to clarify the desired format for the AI.  

---

### 6. Debugging Parameters  

#### **Purpose**  

- Logs the parameters being sent to the AI for debugging purposes.  

#### **Code**  

- **`json.dumps(params, indent=2):`**  

  - Formats the parameters dictionary as a readable JSON string with 2-space indentation.  

---

### 7. Communicating with the AI Model  

#### **Purpose**  

- Sends the query and schema to the **`ollama.chat`** function to interact with the specified AI model.  

#### **Code**  

- **`messages:`**  

  - A list of messages simulating a conversation.  

  - The role of **`"user"`** indicates the input is a user query.  

- **`format="json":`**  

  - Specifies that the response should be formatted as **JSON**.  

---

### 8. Processing the Response  

#### **Code**  

- **`response.message.content:`**  

  - Retrieves the raw content of the model's response.  

- **`json.loads(response_json):`**  

  - Parses the **JSON** string into a Python dictionary.  

- **`response_model.model_validate(response_obj):`**  

  - Validates and converts the parsed **JSON** response into a structured **Pydantic** object.  

---

### 9. Exception Handling  

#### **Purpose**  

- Catches errors that may occur during execution, such as invalid **JSON** or schema mismatches.  

#### **Code**  

- **`traceback.print_exc():`**  

  - Prints a detailed stack trace to help diagnose the issue.  

---

## Conclusion  

- This code integrates structured data validation (**Pydantic**) with an AI model interface (**ollama.chat**).  

- It provides a robust way to interact with the model while ensuring the output adheres to a predefined schema, enhancing reliability and usability in applications.  

- Debugging and error-handling mechanisms ensure smooth operation even when issues arise.  


In [45]:
from typing import Union
from pydantic import BaseModel
import json
import traceback

DEFAULT_SYSTEM_PROMPT = (
    "You are an intelligent assistant. You are helping the user with their query."
)
DEFAULT_TEMPERATURE = 0.5
DEFAULT_MAX_TOKENS = 100
DEFAULT_OLLAMA_MODEL = "phi4"
DEFAULT_VERBOSE = True
DEFAULT_DEBUG = True


def build_dummy_pydantic_object(schema: BaseModel) -> BaseModel:
    """
    Build a dummy Pydantic object using the given schema.

    Args:
      schema: The Pydantic schema to build the object from

    Returns:
      BaseModel: The dummy Pydantic object
    """
    return schema()


def generate_object(
    prompt: str,
    response_model: BaseModel,
    system=DEFAULT_SYSTEM_PROMPT,
    model=DEFAULT_OLLAMA_MODEL,
    temperature=DEFAULT_TEMPERATURE,
    max_tokens=DEFAULT_MAX_TOKENS,
    debug=DEFAULT_DEBUG,
    verbose=DEFAULT_VERBOSE,
) -> Union[BaseModel, None]:
    """Generates an object using the OpenAI API and given response model."""
    try:
        if verbose or debug:
            print(f"Generating object for prompt: {prompt}")

        prompt_with_structured_output = f"""
            Prompt: {prompt} 
            SCHEMA: {build_dummy_pydantic_object(response_model).model_dump_json()}
            RESPOND IN JSON FORMAT
        """

        if debug:
            params = {
                "prompt": prompt_with_structured_output,
                "system": system,
                "temperature": temperature,
                "max_tokens": max_tokens,
                "model": model,
            }
            params = json.dumps(params, indent=2)
            print(f"Params: {params}")

        response = ollama.chat(
            model=model,
            messages=[
                # {"role": "system", "content": system},
                {"role": "user", "content": prompt},
            ],
            format="json",
        )

        response_json = response.message.content  # Get the response content
        print(response_json)
        response_obj = json.loads(response_json)
        response_structured = response_model.model_validate(response_obj)

        if verbose or debug:
            print("Object generated successfully. 🎉")

        if debug:
            print(f"EasyLLM Response: {response_json}")
        return response_structured
    except Exception as e:
        print(f"Failed to generate object. Error: {str(e)}")
        if debug:
            traceback.print_exc()
        return None

## Step 5: Validating JSON Using Pydantic Models

This block uses Pydantic to define a schema for a joke object, including a joke and its author. The assistant is prompted to generate a joke that adheres to this schema. The structured JSON response is validated and displayed.

In [ ]:
from pydantic import BaseModel


class Joke(BaseModel):
    joke: str = (
        "Why did the scarecrow win an award? Because he was outstanding in his field."
    )
    author: str = "Unknown"


response = generate_object("Tell me a joke", response_model=Joke)

print(response)

## Step 6: Requesting and Validating AI-Generated Joke with Pydantic Schema  

---  

### Importing Necessary Libraries  

- **`import ollama:`**  

  - Imports the **`ollama`** library, which is used to interact with an AI model for natural language processing tasks.  

- **`from pydantic import BaseModel:`**  

  - Imports the **`BaseModel`** class from the **`Pydantic`** library.  

  - **`BaseModel`** helps in defining data schemas and validating data.  

---  

### Defining the Joke Model  

- **`class Joke(BaseModel):`**  

  - Defines a **Pydantic** model class called **`Joke`** that inherits from **`BaseModel`**.  

  - This model will represent the structure of the joke response from the AI.  

- **`joke: str`** and **`author: str:`**  

  - These are the attributes of the **`Joke`** class.  

  - **`joke`** is a string containing the joke text.  

  - **`author`** is a string containing the name of the author of the joke.  

---  

### Constructing the Prompt  

- **`prompt = f""":`**  

  - Defines a multi-line formatted string (**f-string**) that will be used to interact with the AI model.  

- **`"Tell me a joke.":`**  

  - The user query instructs the AI to tell a joke.  

- **`Respond with the schema: {Joke.model_json_schema()}:`**  

  - The AI is instructed to respond in a specific format, which is defined by the **`Joke`** model.  

  - **`Joke.model_json_schema()`** dynamically inserts the **JSON schema** of the **Joke** model (i.e., the structure of the joke response) into the prompt.  

- **`Include only the keys and values and not the schema itself.:`**  

  - Instructs the AI to return only the values (**joke** and **author**), excluding the schema structure.  

---  

### Generating the Response Using the generate_object Function  

- **`response = generate_object(prompt, response_model=Joke):`**  

  - Calls the **`generate_object`** function (defined in the previous code) to send the prompt to the AI model.  

  - The **`response_model=Joke`** argument ensures the response is validated against the **Joke** model.  

---  

### Printing the Response  

- **`print(response):`**  

  - Prints the structured response returned by the **`generate_object`** function.  

  - The response should be a **Joke** object containing the joke and the author.  

---  

### Conclusion  

- This code constructs a structured prompt to request a joke from the AI and formats the response according to a predefined **Pydantic** schema (**Joke**).  

- By using the **`generate_object`** function, the response is validated and ensures that only the required data (the joke and author) is returned in a clean and structured format.  


In [ ]:
import ollama

from pydantic import BaseModel


class Joke(BaseModel):
    joke: str
    author: str


prompt = f"""
"Tell me a joke. 
Respond with the schema: {Joke.model_json_schema()}"
Include only the keys and values and not the schema itself.
"""

response = generate_object(prompt, response_model=Joke)

print(response)

## Step 7: Generating and Validating AI-Generated Joke with Schema Enforcement

---  

### Importing Necessary Libraries  

- **`import ollama:`**  

  - Imports the **`ollama`** library, which is used for interacting with AI models like **phi4** for natural language processing tasks.  

- **`from pydantic import BaseModel:`**  

  - Imports the **`BaseModel`** class from the **Pydantic** library.  

  - **`BaseModel`** helps in defining schemas and validating data for structured responses.  

---  

### Defining the Joke Model  

- **`class Joke(BaseModel):`**  

  - Defines a **Pydantic** model class called **Joke**, inheriting from **BaseModel**.  

  - The **Joke** model will structure the response containing a joke and its author.  

- **`joke: str = "A joke"`** and **`author: str = "An author":`**  

  - These are default attributes of the **Joke** class.  

  - **`joke`** holds a string that represents the joke text, with a default value **"A joke"**.  

  - **`author`** holds a string for the author of the joke, with a default value **"An author"**.  

---  

### Building a Dummy Pydantic Object  

- **`def build_dummy_pydantic_object(schema: BaseModel) -> BaseModel:`**  

  - Defines a helper function **`build_dummy_pydantic_object`** that creates an instance of a **Pydantic** model (in this case, **Joke**) with dummy values.  

- **Function Explanation:**  

  - The function takes a **schema** (which is a **Pydantic** model) as an argument.  

  - It returns an instance of the provided schema initialized with its default values.  

---  

### Constructing the Prompt  

- **`prompt = f""":`**  

  - Defines a formatted string (**f-string**) to create a structured prompt for the AI.  

- **`"Tell me a joke.":`**  

  - The main instruction for the AI to generate a joke.  

- **`Respond with the schema — Example object: {build_dummy_pydantic_object(Joke).model_dump_json()}:`**  

  - This part asks the AI to respond in the format defined by the **Joke** schema.  

  - **`build_dummy_pydantic_object(Joke).model_dump_json()`** provides an example object in **JSON** format, which is used to show the AI how to structure the response.  

- **`Include only the keys and values and not the schema itself.:`**  

  - Instructs the AI to return just the values for the **joke** and **author** fields, without the schema definition.  

- **`Respond strictly in the given schema format.:`**  

  - Ensures that the AI’s response adheres strictly to the structure of the **Joke** schema.  

- **`Respond only with valid JSON and don't include anything else.:`**  

  - Further enforces the requirement for the AI to return a clean, well-formed **JSON** response, without any extraneous information.  

---  

### Generating the Response Using the ollama.chat Function  

- **`response = ollama.chat("phi4", messages=[{"role": "user", "content": prompt}]):`**  

  - Sends the structured prompt to the **phi4** AI model using **ollama.chat**.  

  - The AI responds with a message formatted according to the instructions provided in the prompt.  

---  

### Printing the Response  

- **`print(response.message.content):`**  

  - Prints the content of the response message returned by the AI.  

  - This will be the joke along with the author's name in **JSON** format, strictly following the schema.  

---  

### Conclusion  

- This code interacts with the **phi4** AI model to generate a joke and its author, ensuring that the response strictly adheres to the predefined **Joke** schema.  

- The response is validated against the schema and returned in a clean **JSON** format, which only includes the joke and author, excluding the schema structure itself.  


In [ ]:
import ollama

from pydantic import BaseModel


class Joke(BaseModel):
    joke: str = "A joke"
    author: str = "An author"


def build_dummy_pydantic_object(schema: BaseModel) -> BaseModel:
    """
    Build a dummy Pydantic object using the given schema.

    Args:
      schema: The Pydantic schema to build the object from

    Returns:
      BaseModel: The dummy Pydantic object
    """
    return schema()


prompt = f"""
"Tell me a joke. 
Respond with the schema — Example object: {build_dummy_pydantic_object(Joke).model_dump_json()}"
Include only the keys and values and not the schema itself.
Respond strictly in the given schema format.
Respond only with valid JSON and don't include anything else.
"""

response = ollama.chat("phi4", messages=[{"role": "user", "content": prompt}])

print(response.message.content)

## Conclusion

This notebook effectively demonstrates how to integrate structured data processing with language models. The approach ensures schema validation, making it suitable for applications like API testing, structured data generation, and schema-driven conversations. The use of Pydantic ensures type safety and consistency, while Ollama's models bring the power of conversational AI.

---

# Thank You for visiting The Hackers Playbook! 🌐

If you liked this research material;

- [Subscribe to our newsletter.](https://thehackersplaybook.substack.com)

- [Follow us on LinkedIn.](https://www.linkedin.com/company/the-hackers-playbook/)

- [Leave a star on our GitHub.](https://www.github.com/thehackersplaybook)

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>